In [1]:
# changed

import numpy as np
#import pandas as pd
import pickle
import time
import os
import copy
import sys
#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import read_image


# models
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import regnet_x_8gf, RegNet_X_8GF_Weights
from torchvision.models import swin_t, Swin_T_Weights
from torchvision.models import wide_resnet50_2, Wide_ResNet50_2_Weights

import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)
print(sys.version)
print(torch.__version__)
print(torchvision.__version__)

cpu
3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
1.12.1
0.13.1


In [2]:
# changed

#wandb.login()
#wandb.init(project="test_project_0", entity="nornir")

#should be terminal inputs:

# choose one model
#model_name = 'convnext_tiny' # last block is called "classifier" 
# model_name = 'efficientnet_v2_s' # last block is called "classifier" 
model_name = 'regnet_x_8gf' # last block is called "fc" 
# model_name = 'swin_t'  # last block is called "head" 
# model_name = 'wide_resnet50_2'  # last block is called "fc" 

attribute = 'all_mass_protest' # or maybe this is a loop. Or both are loop?

# ---------------------------------------

hyperparameters = {
"model_name" : model_name,
"attribute" : attribute,
"learning_rate": 0.0001,
"weight_decay" : 0.05,
'betas' : (0.9, 0.999),
"classes" : 1,
"epochs": 2,
"batch_size": 8 # efficientnet_v2_s can max do 32 before running our of mem.
}

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, attribute_dict, attribute, img_dir, train = True, transform=None, target_transform=None):

        # need to handle nan : do it in the df to dict..

        n_img = n_img = len(attribute_dict[f'{attribute}_ens_mean'])
        n_train = int(n_img * 0.95)

        if train == True:

            self.img_labels = np.array(list(zip(attribute_dict['img'], attribute_dict[f'{attribute}_ens_mean'])))[:n_train] # which is not a label but a score.. # you do not handles nans right now... 
            self.img_std = np.array(list(zip(attribute_dict['img'], attribute_dict[f'{attribute}_ens_std'])))[:n_train] # you do not handles nans right now...
        
        elif train == False:
            self.img_labels = np.array(list(zip(attribute_dict['img'], attribute_dict[f'{attribute}_ens_mean'])))[n_train:] # which is not a label but a score.. # you do not handles nans right now... 
            self.img_std = np.array(list(zip(attribute_dict['img'], attribute_dict[f'{attribute}_ens_std'])))[n_train:]

        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx, 0])
        image = read_image(img_path)
        label = np.float32(self.img_labels[idx, 1]) #it is turned to a str above so we turn it back here
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [4]:
def get_models():

    weight_dict = {'convnext_tiny': ConvNeXt_Tiny_Weights.DEFAULT,
                'efficientnet_v2_s' : EfficientNet_V2_S_Weights.DEFAULT,
                'regnet_x_8gf' : RegNet_X_8GF_Weights.DEFAULT,
                'swin_t' : Swin_T_Weights.DEFAULT,
                'wide_resnet50_2' : Wide_ResNet50_2_Weights.DEFAULT}

    model_dict = {'convnext_tiny': convnext_tiny(weights = weight_dict['convnext_tiny']).to(device),
                'efficientnet_v2_s' : efficientnet_v2_s(weights = weight_dict['efficientnet_v2_s']).to(device),
                'regnet_x_8gf' : regnet_x_8gf(weights = weight_dict['regnet_x_8gf']).to(device),
                'swin_t' : swin_t(weights = weight_dict['swin_t']).to(device),
                'wide_resnet50_2' : wide_resnet50_2(weights = weight_dict['wide_resnet50_2']).to(device)}

    return(weight_dict, model_dict)

In [5]:
# change last layer
def change_head(model_name, model, num_classes):

    if model_name == 'convnext_tiny':
        model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes).to(device)
        print(f'new head: {model.classifier[2]}')

    elif model_name == 'efficientnet_v2_s':
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes).to(device)
        print(f'new head: {model.classifier[1]}')

    elif model_name == 'regnet_x_8gf':
        model.fc = nn.Linear(model.fc.in_features, num_classes).to(device)
        print(f'new head: {model.fc}')

    elif model_name == 'swin_t':
        model.head = nn.Linear(model.head.in_features, num_classes).to(device)
        print(f'new head: {model.head}')

    elif model_name == 'wide_resnet50_2':
        model.fc = nn.Linear(model.fc.in_features, num_classes).to(device)
        print(f'new head: {model.fc}')

    else:
        print('Unddefined model name...')


In [6]:
# data loader
def make_loader(batch_size, weights, attribute):

    # this is the thiong that has to change for RA...
    # need to get attribute

    #Should be in config
    data_transforms = {
    'train': transforms.Compose([weights.transforms(), transforms.RandomHorizontalFlip(p=0.5), transforms.RandomRotation(degrees=(0, 45)), transforms.ColorJitter(brightness=.5, hue=.2)]), 
    'test': transforms.Compose([weights.transforms()])
    }

    # Going into make loader -------------------------------

    dict_dir = '/home/simon/Documents/Bodies/data/RA/dfs/' #local
    img_dir = '/media/simon/Seagate Expansion Drive/images_spanner' #local

    #dict_dir = '/home/projects/ku_00017/data/raw/bodies/RA_annotations/' # computerome
    #img_dir = '/home/projects/ku_00017/data/raw/bodies/images_spanner' # computerome

    with open(f'{dict_dir}ra_ens_annotated_dict.pkl', 'rb') as file:
        attribute_dict = pickle.load(file)

    dataloaders = {}
    image_datasets = {}
    
    image_datasets['train'] = CustomImageDataset(attribute_dict, attribute, img_dir, train=True , transform=data_transforms['train'])
    dataloaders['train'] = DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True)

    image_datasets['test'] = CustomImageDataset(attribute_dict, attribute, img_dir, train=False , transform=data_transforms['test'])
    dataloaders['test'] = DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=True)

    #image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    #dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    #class_names = image_datasets['train'].classes

    return dataloaders, dataset_sizes #class_names # what did you use class names for?

In [7]:
def make(config, model_name):

    # Make the model
    weight_dict, model_dict = get_models()

    #Choose model and wieghts
    weights = weight_dict[model_name] # you need these later right as they hold the appropiate data transforamtions
    model = model_dict[model_name].to(device)
    # wandb.watch(model)

    # new model head for for retraining
    change_head(model_name, model, config['classes'])

    # re-train all parameters
    for param in list(model.parameters()):
        param.requires_grad = True
    
    # Make the data
    dataloaders, dataset_sizes = make_loader(batch_size=config['batch_size'],  weights = weights, attribute = config['attribute'])
    
    # Make the loss and optimizer
    #criterion = nn.CrossEntropyLoss()
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=config['learning_rate'], weight_decay = config['weight_decay'], betas = config['betas'])

    return model, criterion, optimizer, dataloaders, dataset_sizes #, class_names

In [8]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    # wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")


In [9]:
def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs.squeeze(), labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss


In [10]:
def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    #wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config['epochs']
    example_ct = 0  # number of examples seen
    batch_ct = 0
    running_loss = 0.0

    for epoch in range(config['epochs']):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            running_loss += loss

            # Report metrics every 20th batch - not running average right now
            if ((batch_ct + 1) % 100) == 0:
                #train_log(loss, example_ct, epoch) # this is the wand part
                train_log(running_loss/100, example_ct, epoch)
                running_loss = 0.0 # reset


In [11]:
def test(model, test_loader):
    model.eval()
    test_criterion = nn.MSELoss()

    # Run the model on some test examples
    with torch.no_grad():
        #correct, total = 0, 0
        total = 0
        RMSE_loss = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            #_, predicted = torch.max(outputs.data, 1)
            RMSE_loss += torch.sqrt(test_criterion(outputs.squeeze().cpu(), labels.cpu()))

            total += 1 #labels.size(0)
            #correct += (predicted == labels).sum().item()

        # print(f"Accuracy of the model on the {total} " +
        #       f"test images: {100 * correct / total}%")
        
        print(f"Average RMSE of the model on the {total} " +
              f"test images: {RMSE_loss / total}%")


        #wandb.log({"test_rmse": RMSE_loss / total})

    # Save the model in the exchangeable ONNX format
    #torch.onnx.export(model, images, "model.onnx")
    #wandb.save("model.onnx")

In [12]:
# access all HPs through wandb.config, so logging matches execution!
config = hyperparameters

model_name = config['model_name']

# make the model, data, and optimization problem
model, criterion, optimizer, dataloaders, dataset_sizes = make(config, model_name)
print(model)

new head: Linear(in_features=1920, out_features=1, bias=True)
RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 80, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(80, 80, kernel_size=(3, 3), str

In [13]:
# # and use them to train the model
# train(model, dataloaders['train'], criterion, optimizer, config)

# # and test its final performance
# test(model, dataloaders['test'])

In [14]:
# BS = 4
model.eval()

test_criterion = nn.MSELoss()

total = 0
RMSE_loss_running = 0
RMSE_list = []

with torch.no_grad():
    for images, labels in dataloaders['test']:
        y = model(images)
        torch.sqrt(test_criterion(y.squeeze(), labels))

        #_, predicted = torch.max(outputs.data, 1)
        
        RMSE_loss = torch.sqrt(test_criterion(y.squeeze(), labels))
        RMSE_list.append(RMSE_loss.detach().numpy().item())
        RMSE_loss_running += RMSE_loss
        
        print(f'Batch RMSE loss: {RMSE_loss}')
        
        total += 1 # labels.size(0) #  1

    print(f"Average RMSE of the model on the {total} " + f"test images: {RMSE_loss_running / total}%")
    print(f"alt version: {np.array(RMSE_list).mean()}")


Batch RMSE loss: 1.0530973672866821
Batch RMSE loss: 1.041982889175415
Batch RMSE loss: 1.1272201538085938
Batch RMSE loss: 0.6610047817230225
Batch RMSE loss: 0.9385876059532166
Batch RMSE loss: 0.6299359798431396
Batch RMSE loss: 1.1221593618392944
Batch RMSE loss: 1.3915281295776367
Batch RMSE loss: 1.025121808052063
Batch RMSE loss: 0.5970933437347412
Batch RMSE loss: 1.4491084814071655
Batch RMSE loss: 0.8849812746047974
Batch RMSE loss: 0.7312242388725281
Batch RMSE loss: 0.675737738609314
Batch RMSE loss: 0.9315090775489807
Batch RMSE loss: 0.7649460434913635
Batch RMSE loss: 0.8738184571266174
Batch RMSE loss: 0.7086728811264038
Batch RMSE loss: 0.7295205593109131
Batch RMSE loss: 0.9991548657417297
Batch RMSE loss: 0.744766891002655
Batch RMSE loss: 1.5776572227478027
Average RMSE of the model on the 22 test images: 0.9390376806259155%
alt version: 0.9390376887538217


In [ ]:
# Batch RMSE loss: 0.6822541952133179


In [75]:
# BS = 8
model.eval()

test_criterion = nn.MSELoss()

total = 0
RMSE_loss_running = 0
RMSE_list = []

for images, labels in dataloaders['test']:
    y = model(images)
    torch.sqrt(test_criterion(y.squeeze(), labels))

    #_, predicted = torch.max(outputs.data, 1)
    
    RMSE_loss = torch.sqrt(test_criterion(y.squeeze(), labels))
    RMSE_list.append(RMSE_loss.detach().numpy().item())
    RMSE_loss_running += RMSE_loss
    
    print(f'Batch RMSE loss: {RMSE_loss}')
    
    total += 1 # labels.size(0) #  1

print(f"Average RMSE of the model on the {total} " + f"test images: {RMSE_loss_running / total}%")
print(f"alt version: {np.array(RMSE_list).mean()}")


Batch RMSE loss: 1.0091129541397095
Batch RMSE loss: 0.9891952872276306
Batch RMSE loss: 1.277721881866455
Batch RMSE loss: 1.022247552871704
Batch RMSE loss: 0.930595874786377
Batch RMSE loss: 0.744383692741394
Batch RMSE loss: 0.9413062334060669
Batch RMSE loss: 0.6356397271156311
Batch RMSE loss: 1.377557396888733
Batch RMSE loss: 0.5771723389625549
Batch RMSE loss: 0.6576986908912659
Batch RMSE loss: 1.154015064239502
Batch RMSE loss: 1.0379843711853027
Batch RMSE loss: 0.3965778350830078
Batch RMSE loss: 0.8038842082023621
Batch RMSE loss: 0.8570382595062256
Batch RMSE loss: 0.9580329060554504
Batch RMSE loss: 0.9573437571525574
Batch RMSE loss: 0.8930890560150146
Batch RMSE loss: 0.9257214069366455
Batch RMSE loss: 1.5151762962341309
Batch RMSE loss: 0.5565876364707947
Average RMSE of the model on the 22 test images: 0.9190038442611694%
alt version: 0.9190037467262961


In [19]:
epochs = 64

for epoch in range(epochs):
    if epoch == epochs-1:
        print(epoch)

63


In [35]:
len(RMSE_list)

43

In [33]:
RMSE_loss.detach().numpy().item()

1.4013563394546509

In [27]:
# BS = 4
model.eval()

test_criterion = nn.MSELoss()

total = 0
RMSE_loss = 0

for images, labels in dataloaders['test']:
    y = model(images)
    torch.sqrt(test_criterion(y.squeeze(), labels))

    #_, predicted = torch.max(outputs.data, 1)
    RMSE_loss += torch.sqrt(test_criterion(y.squeeze(), labels))
    
    
    total += 1 #labels.size(0) #  1

print(f"Average RMSE of the model on the {total} " + f"test images: {RMSE_loss / total}%")


Average RMSE of the model on the 43 test images: 0.9227243661880493%
